# PSUPR / PRMLS CA2

### Team
Kenneth Goh Chia Wei : A0198544N  
Tan Heng Han : A0198502B  
Raymond Ng Boon Cheong : A0198543R

### CA2 Test Script
Perform sentiment analysis on tweets from Apple event

In [4]:
import pandas as pd
import lxml
import re
import time

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

### Create a function for cleaning up tweets

In [5]:
def tweet_cleaner(text):
    # Init WordPunctTokenizer
    tok = WordPunctTokenizer()
    # create reg expression
    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2))
    # Use beautiful soup to decode html to text
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    # strip text using regular expression
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    # strip to letters only
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    # remove retweets handle
    removed_rt = re.sub('RT ','',letters_only)
    # set to lower case for all characters
    lower_case = removed_rt.lower()
    # Use tokenize to remove whitespace created by letters_only above
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

### Load Apple event data from csv

In [6]:
file = '..\\dataset\\tweets_appleEvent.csv'
df = pd.read_csv(file)

In [7]:
df.shape

(1000, 14)

In [8]:
df.head()

,text,weekday,month,day,hour,has_hashtag,has_url,fav_count,is_reply,retweet_count,followers,following,user,screen_name
0,RT @timmaughan: Back in 2014 I visited Inner M...,Tue,Sep,10,16,0,0,0,0,4,3086,1288,Roderick McKie Esquire,rodmckie
1,RT @beebomco: iPhone 11 launches tonight. You ...,Tue,Sep,10,16,1,0,0,0,1716,14,90,God Kizaru🐝🇩🇴🍑,GodKizaruRD
2,RT @CNET: Tune into the #AppleEvent today at:\...,Tue,Sep,10,16,1,0,0,0,84,418,2700,on/off ™,EstherLamarr
3,RT @BraJinglez: Apple has added a new payment ...,Tue,Sep,10,16,1,0,0,0,5943,848,352,Yemi Stark,yemiscofield
4,RT @mauobenol: iPhone 11 launches tomorrow! An...,Tue,Sep,10,16,1,0,0,0,4094,48,69,𝗸𝗮𝗶𝗹𝗲𝗲。,_seoularr


In [9]:
# Keep only tweets and its label
text = df['text']
text.head()

0    RT @timmaughan: Back in 2014 I visited Inner M...
1    RT @beebomco: iPhone 11 launches tonight. You ...
2    RT @CNET: Tune into the #AppleEvent today at:\...
3    RT @BraJinglez: Apple has added a new payment ...
4    RT @mauobenol: iPhone 11 launches tomorrow! An...
Name: text, dtype: object

### Clean up tweets

In [10]:
%%time
cleaned_tweets = []
for tweet in text:
    cleaned_tweets.append(tweet_cleaner(tweet))
print(cleaned_tweets[:5])
print(f'Tweets total: {len(cleaned_tweets)}')

['back in i visited inner mongolia and this toxic run off lake that s a byproduct of rare earth refining want to help', 'iphone launches tonight you ready iphone iphonexi appleevent apple', 'tune into the appleevent today at am pdt pm cdt pm edt pm bst pm ist wednesday', 'apple has added a new payment option for iphone appleevent', 'iphone launches tomorrow and i still have an iphone s appleevent']
Tweets total: 1000
Wall time: 247 ms


### Rejoin all tweets and drop duplicate tweets

In [11]:
appleTweets = pd.DataFrame({'text': cleaned_tweets})
appleTweets.drop_duplicates(subset='text', inplace=True, keep='first')

In [12]:
appleTweets.shape

(328, 1)

In [13]:
appleTweets.head()

,text
0,back in i visited inner mongolia and this toxi...
1,iphone launches tonight you ready iphone iphon...
2,tune into the appleevent today at am pdt pm cd...
3,apple has added a new payment option for iphon...
4,iphone launches tomorrow and i still have an i...


### Load pre-train model

In [14]:
model = load_model('train_model')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Import trained tokenizer

In [15]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [16]:
labels = ['happy', 'sad', 'angry']

def label_sentiment(scores):
    maxScore = 0
    maxIdx = 0
    for i in range(len(scores)):
        if scores[i] > maxScore:
            maxScore = scores[i]
            maxIdx = i
    return maxIdx

def predict(text):
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300)
    # Predict
    scores = model.predict([x_test])[0]
    # Decode sentiment
    lblIdx = label_sentiment(scores)
    return labels[lblIdx], scores

### Perform sentiment analysis on tweets

In [19]:
%%time
for txt in appleTweets.text:
    lbl, scores = predict(txt)
    print(f'Tweet: {txt}')
    print(f'Predicted Sentiment: {lbl}, Scores: {scores}\n')

Tweet: back in i visited inner mongolia and this toxic run off lake that s a byproduct of rare earth refining want to help
Predicted Sentiment: angry, Scores: [0.33241805 0.31148502 0.35609692]

Tweet: iphone launches tonight you ready iphone iphonexi appleevent apple
Predicted Sentiment: angry, Scores: [0.33072525 0.30553904 0.3637357 ]

Tweet: tune into the appleevent today at am pdt pm cdt pm edt pm bst pm ist wednesday
Predicted Sentiment: happy, Scores: [0.6092634  0.22173415 0.16900244]

Tweet: apple has added a new payment option for iphone appleevent
Predicted Sentiment: happy, Scores: [0.35939816 0.32045642 0.32014543]

Tweet: iphone launches tomorrow and i still have an iphone s appleevent
Predicted Sentiment: happy, Scores: [0.34559858 0.32390696 0.33049443]

Tweet: rawkz i m confused did apple just launched a new gas cooker or is this the iphone appleevent
Predicted Sentiment: happy, Scores: [0.3501239  0.32317123 0.32670483]

Tweet: cupertino calling join us today at a m p